In [1]:
!activate PythonGPU
import numpy as np
from scipy.stats import skewnorm, skew
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, accuracy_score

def simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew, random_state=1234):
    #The multivariate skew normal number generator
    def rng(mu, sigma, skew, n=1):
        k = len(mu)
        if not (k == len(sigma) and k ==len(skew)): 
            raise Exception("Mu, Sigma and Skew should be same length")

        data = np.zeros((int(n),k))

        for i in range(k):
            data[:,i] = skewnorm.rvs(skew[i], loc=mu[i], scale=sigma[i], size=int(n)) 

        return data
    
    if(np.sum(classes) != 1):
        raise Exception("Classes dont sum up to 1")
        
    n_classes = len(classes)
    
    np.random.seed(random_state)
    sigma = (max_sigma * np.random.rand(1,n_vars))[0]
    
    np.random.seed(random_state)
    skew = ((2 * max_skew  * np.random.rand(1, n_vars)) - max_skew)[0]
    
    np.random.seed(random_state)
    mu = (2 *  max_mu * np.random.rand(n_classes, n_vars)) - max_mu
    
    n_obs_class = np.round(np.dot(classes,n))
    
    data = np.zeros((int(np.sum(n_obs_class)),n_vars+1))
    for i in range(n_classes):
        #calculate indexes
        start = int(np.sum(n_obs_class[0:i]))
        end = int(np.sum(n_obs_class[0:i+1]))
        
        #set the data
        data[start:end,0] = i
        data[start:end,1:] = rng(mu[i,:], sigma, skew, n_obs_class[i])
        
    X = data[:,1:]
    y = data[:,0]
        
    X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42,
    stratify=y)
    
    return X_train, X_test, y_train, y_test

In [2]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def classify_lda(X_train, X_test, y_train, y_test, priors, plot=False):
    lda = LinearDiscriminantAnalysis(priors=priors)
    X_lda = lda.fit_transform(X_train, y_train)

    predictions = lda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("LDA Test accuracy "+ str(accuracy))
    print(predictions)

    if plot:    
        plt.xlabel('LD1')
        plt.ylabel('LD2')
        plt.scatter(
            X_lda[:,0],
            X_lda[:,1],
            c=y_train,
            cmap='Accent',
        )
        
    return {"method": "LDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": lda}

In [3]:
#Quadratic
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify_qda(X_train, X_test, y_train, y_test, priors):
    qda = QuadraticDiscriminantAnalysis(priors=priors)
    X_qda = qda.fit(X_train, y_train)



    predictions = qda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("QDA Test accuracy "+ str(accuracy))

    return {"method": "QDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": qda}

In [4]:
from sklearn.linear_model import LogisticRegression

def classify_logit(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0, solver='lbfgs',
                             multi_class='multinomial').fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Logistic Test accuracy "+ str(accuracy))
    
    return {"method": "Logit", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": clf}

In [5]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

def classify_knn(X_train, X_test, y_train, y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(X_train, y_train)

    predictions = knn.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("KNN-"+str(n_neighbors)+" Test accuracy "+ str(accuracy))
    
    return {"method": "KNN-"+str(n_neighbors), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": knn}

In [6]:
#Naive bayes
from sklearn.naive_bayes import GaussianNB

def classify_naivebayes(X_train, X_test, y_train, y_test, priors):
    NB = GaussianNB(priors)
    NB.fit(X_train, y_train)
    
    predictions = NB.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("Naive Bayes Test accuracy "+ str(accuracy))
    
    return {"method": "Naive Bayes", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": NB}

In [7]:
#SVM
from sklearn.svm import LinearSVC

def classify_svm(X_train, X_test, y_train, y_test):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    
    predictions = svm.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("SVM Test accuracy "+ str(accuracy))
    
    return {"method": "SVM", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": svm}

In [8]:
from tensorflow.keras import layers
from tensorflow import keras

def classify_neuralnet(X_train, X_test, y_train, y_test, n_vars, n_classes, depth=1, nodes=10, epochs=20):
    inputs = keras.Input(shape=(n_vars,), name='obs')
    x = layers.Dense(nodes, activation='relu')(inputs)
    
    if(depth>1):
        for i in range(depth-1):
            x = layers.Dense(nodes, activation='relu')(x)
            
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='Dataset')

    display(model.summary())


    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=64,
                        epochs=epochs,
                        validation_split=0.2)

    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    print(predictions)

    accuracy = accuracy_score(y_test, predictions)
    print("Neural Network Test accuracy "+ str(accuracy))
    
    return {"method": "Net "+"-".join([str(nodes) for i in range(depth)])+ " E"+str(epochs), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": model}

In [9]:
from datetime import datetime
import pandas as pd
import time


np.random.seed(12345)

timelabel =  datetime.now().strftime("%H-%M-%S - %d-%m-%Y")


configs = [{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 500,
    "max_mu": 2,
    "max_sigma": 2,
    "max_skew": 1
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.05, 0.05, 0.05, 0.85],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 50,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.125 for x in range(8)],
    "n_vars": 5,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 1000000,
    "max_mu": 1,
    "max_sigma": 5,
    "max_skew": 5
}]



columns = ['method', 'accuracy','predictions', "model", "config"] + list(configs[0].keys())
results = pd.DataFrame(columns=columns)
results.style.format({
    'accuracy': '{:,.3%}'.format
})


#run analysis
for i, c in enumerate(configs):
    X_train, X_test, y_train, y_test = simulate_data(c["classes"], c["n_vars"], c["n"], c["max_mu"], c["max_sigma"], c["max_skew"], 1234)
    
    c_n = i+1
    
    lda = classify_lda(X_train, X_test, y_train, y_test, c["classes"], False)                                     
    results = results.append({**lda, **c, "config":c_n},ignore_index=True)
    
    qda = classify_qda(X_train, X_test, y_train, y_test, c["classes"])                                     
    results = results.append({**qda, **c, "config":c_n},ignore_index=True)
    
    logit = classify_logit(X_train, X_test, y_train, y_test)                                     
    results = results.append({**logit, **c, "config":c_n},ignore_index=True)
    
    for k in [5,10,50,100]:
        knn = classify_knn(X_train, X_test, y_train, y_test, k)
        results = results.append({**knn, **c, "config":c_n},ignore_index=True)

    bayes = classify_naivebayes(X_train, X_test, y_train, y_test, c["classes"])
    results = results.append({**bayes, **c, "config":c_n},ignore_index=True)
    
    svm = classify_svm(X_train, X_test, y_train, y_test)
    results = results.append({**svm, **c, "config":c_n},ignore_index=True)
    
    for n in [{"d":1,"n":len(c["classes"]), "e":25}, {"d":4,"n":30, "e":10}, {"d":4,"n":50, "e":50}]:
        neuralnet = classify_neuralnet(X_train, X_test, y_train, y_test, c["n_vars"], len(c["classes"]),  depth=n["d"], nodes=n["n"], epochs=n["e"])                                 
        results = results.append({**neuralnet, **c, "config":c_n},ignore_index=True)
    
    print("Results after config "+str(c_n)+" of "+str(len(configs)))

    results.sort_values(by='accuracy', ascending=False, inplace=True)
        
    display(results.style.format({
    'accuracy': '{:,.3%}'.format
    }))
    

    #saving results to file
    results.drop(columns=['model']).to_pickle("./results/config "+str(i+1)+" of "+str(len(configs))+" "+timelabel+".pkl")

LDA Test accuracy 0.7515151515151515
[0. 2. 1. 1. 2. 2. 3. 2. 0. 1. 2. 0. 1. 2. 2. 3. 0. 1. 2. 1. 2. 3. 1. 0.
 2. 3. 0. 2. 3. 3. 2. 3. 1. 3. 1. 1. 3. 3. 3. 2. 3. 1. 2. 1. 0. 2. 0. 2.
 2. 1. 3. 1. 0. 1. 1. 3. 2. 1. 0. 1. 0. 0. 0. 1. 1. 3. 0. 1. 1. 1. 3. 2.
 2. 2. 2. 1. 1. 3. 0. 1. 3. 3. 0. 2. 1. 0. 1. 3. 2. 0. 2. 0. 0. 2. 3. 0.
 1. 0. 1. 2. 2. 3. 0. 0. 0. 0. 3. 3. 0. 2. 1. 3. 3. 2. 0. 2. 3. 2. 0. 3.
 0. 2. 3. 0. 3. 3. 0. 2. 1. 0. 3. 3. 1. 1. 3. 1. 1. 2. 2. 3. 2. 3. 3. 3.
 1. 1. 0. 3. 2. 0. 3. 3. 3. 1. 0. 3. 2. 3. 0. 3. 1. 2. 1. 3. 2.]
QDA Test accuracy 0.7393939393939394
Logistic Test accuracy 0.7333333333333333
KNN-5 Test accuracy 0.7090909090909091
KNN-10 Test accuracy 0.7151515151515152
KNN-50 Test accuracy 0.696969696969697
KNN-100 Test accuracy 0.6727272727272727
Naive Bayes Test accuracy 0.7575757575757576
SVM Test accuracy 0.703030303030303
Model: "Dataset"
_________________________________________________________________
Layer (type)                 Output Shape              Par

None

Train on 268 samples, validate on 67 samples
Epoch 1/25
268/268 [==============================] - 2s 6ms/sample - loss: 1.7188 - accuracy: 0.0000e+00 - val_loss: 1.7594 - val_accuracy: 0.0000e+00
Epoch 2/25
268/268 [==============================] - 0s 185us/sample - loss: 1.6907 - accuracy: 0.0075 - val_loss: 1.7339 - val_accuracy: 0.0000e+00
Epoch 3/25
268/268 [==============================] - 0s 127us/sample - loss: 1.6712 - accuracy: 0.0000e+00 - val_loss: 1.7154 - val_accuracy: 0.0000e+00
Epoch 4/25
268/268 [==============================] - 0s 138us/sample - loss: 1.6556 - accuracy: 0.0037 - val_loss: 1.6994 - val_accuracy: 0.0000e+00
Epoch 5/25
268/268 [==============================] - 0s 129us/sample - loss: 1.6419 - accuracy: 0.0037 - val_loss: 1.6830 - val_accuracy: 0.0000e+00
Epoch 6/25
268/268 [==============================] - 0s 135us/sample - loss: 1.6280 - accuracy: 0.0037 - val_loss: 1.6678 - val_accuracy: 0.0000e+00
Epoch 7/25
268/268 [=============================

None

Train on 268 samples, validate on 67 samples
Epoch 1/10
268/268 [==============================] - 1s 3ms/sample - loss: 1.3622 - accuracy: 0.2948 - val_loss: 1.3029 - val_accuracy: 0.4478
Epoch 2/10
268/268 [==============================] - 0s 175us/sample - loss: 1.2976 - accuracy: 0.4590 - val_loss: 1.2523 - val_accuracy: 0.5075
Epoch 3/10
268/268 [==============================] - 0s 153us/sample - loss: 1.2558 - accuracy: 0.4552 - val_loss: 1.2055 - val_accuracy: 0.5373
Epoch 4/10
268/268 [==============================] - 0s 157us/sample - loss: 1.2080 - accuracy: 0.5149 - val_loss: 1.1547 - val_accuracy: 0.5373
Epoch 5/10
268/268 [==============================] - 0s 134us/sample - loss: 1.1629 - accuracy: 0.5634 - val_loss: 1.0992 - val_accuracy: 0.6269
Epoch 6/10
268/268 [==============================] - 0s 142us/sample - loss: 1.1086 - accuracy: 0.5784 - val_loss: 1.0523 - val_accuracy: 0.5970
Epoch 7/10
268/268 [==============================] - 0s 138us/sample - loss: 1.0

...

None

Train on 536000 samples, validate on 134000 samples
Epoch 1/10
536000/536000 [==============================] - 35s 66us/sample - loss: 0.6067 - accuracy: 0.6635 - val_loss: 0.6035 - val_accuracy: 0.6639
Epoch 2/10
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6049 - accuracy: 0.6649 - val_loss: 0.6045 - val_accuracy: 0.6648
Epoch 3/10
536000/536000 [==============================] - 33s 61us/sample - loss: 0.6050 - accuracy: 0.6646 - val_loss: 0.6030 - val_accuracy: 0.6656
Epoch 4/10
536000/536000 [==============================] - 34s 63us/sample - loss: 0.6051 - accuracy: 0.6647 - val_loss: 0.6042 - val_accuracy: 0.6647
Epoch 5/10
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6055 - accuracy: 0.6644 - val_loss: 0.6035 - val_accuracy: 0.6659
Epoch 6/10
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6059 - accuracy: 0.6645 - val_loss: 0.6037 - val_accuracy: 0.6657
Epoch 7/10
536000/536000 [==========

None

Train on 536000 samples, validate on 134000 samples
Epoch 1/50
536000/536000 [==============================] - 35s 66us/sample - loss: 0.6071 - accuracy: 0.6628 - val_loss: 0.6031 - val_accuracy: 0.6665
Epoch 2/50
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6054 - accuracy: 0.6645 - val_loss: 0.6048 - val_accuracy: 0.6659
Epoch 3/50
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6055 - accuracy: 0.6649 - val_loss: 0.6037 - val_accuracy: 0.6639
Epoch 4/50
536000/536000 [==============================] - 33s 62us/sample - loss: 0.6056 - accuracy: 0.6650 - val_loss: 0.6041 - val_accuracy: 0.6656
Epoch 5/50
536000/536000 [==============================] - 34s 64us/sample - loss: 0.6060 - accuracy: 0.6642 - val_loss: 0.6032 - val_accuracy: 0.6655
Epoch 6/50
536000/536000 [==============================] - 35s 65us/sample - loss: 0.6065 - accuracy: 0.6648 - val_loss: 0.6061 - val_accuracy: 0.6662
Epoch 7/50
536000/536000 [==========

,method,accuracy,predictions,model,config,classes,n_vars,n,max_mu,max_sigma,max_skew
0,Net 50-50-50-50 E50,100.000%,[2 3 1 ... 1 1 3],,4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
1,KNN-5,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
2,KNN-100,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=100, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
3,Logit,100.000%,[2. 3. 1. ... 1. 1. 3.],"LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='multinomial', n_jobs=None, penalty='l2', random_state=0, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
4,LDA,100.000%,[2. 3. 1. ... 1. 1. 3.],"LinearDiscriminantAnalysis(n_components=None, priors=[0.25, 0.25, 0.25, 0.25], shrinkage=None, solver='svd', store_covariance=False, tol=0.0001)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
5,KNN-10,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=10, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
6,QDA,100.000%,[2. 3. 1. ... 1. 1. 3.],"QuadraticDiscriminantAnalysis(priors=array([0.25, 0.25, 0.25, 0.25]), reg_param=0.0, store_covariance=False, store_covariances=None, tol=0.0001)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
7,Net 4 E25,100.000%,[2 3 1 ... 1 1 3],,4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
8,Naive Bayes,100.000%,[2. 3. 1. ... 1. 1. 3.],"GaussianNB(priors=[0.25, 0.25, 0.25, 0.25], var_smoothing=1e-09)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
9,SVM,100.000%,[2. 3. 1. ... 1. 1. 3.],"LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000, multi_class='ovr', penalty='l2', random_state=None, tol=0.0001, verbose=0)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5


Making tables for the report

In [14]:
import pandas as pd

def bold(data):
    attr = 'font-weight: {}'.format("700")
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    is_max = data == data.max()
    return [attr if v else "" for v in is_max]
    
def boldlatex(data):
    #remove % and cast to float
    return list(map(lambda x: r"\textbf{"+x+"}" if x==data.max() else x, data))
   
table = results.copy()
table['accuracy'] = pd.Series(["{0:.2f}%".format(val * 100) for val in table['accuracy']], index = table.index)
table.loc[table.method == "Net 2 E25", "method"] = "Net N_C E25"
table.loc[table.method == "Net 4 E25", "method"] = "Net N_C E25"
table.loc[table.method == "Net 8 E25", "method"] = "Net N_C E25"

table = table.pivot(index='method', columns='config', values='accuracy').sort_values(by='method')
display(table.style.apply(bold))
latex = table.apply(boldlatex).to_latex()
print(latex)

config,1,2,3,4,5,6,7,8,9,10,11
method,,,,,,,,,,,
KNN-10,71.52%,89.12%,97.12%,100.00%,27.15%,30.05%,33.70%,16.83%,56.50%,63.76%,63.72%
KNN-100,67.27%,88.52%,95.76%,100.00%,30.55%,33.25%,35.34%,19.33%,58.50%,66.02%,66.24%
KNN-5,70.91%,88.70%,97.24%,100.00%,28.12%,28.68%,32.69%,15.94%,54.73%,61.50%,61.65%
KNN-50,69.70%,88.94%,96.48%,100.00%,30.76%,32.51%,35.12%,18.91%,57.82%,65.85%,65.87%
LDA,75.15%,90.79%,98.24%,100.00%,34.79%,34.65%,34.48%,20.79%,57.71%,65.33%,65.48%
Logit,73.33%,90.64%,98.30%,100.00%,35.06%,34.71%,34.51%,20.72%,57.81%,65.50%,65.61%
Naive Bayes,75.76%,90.55%,98.21%,100.00%,34.55%,34.69%,34.42%,20.68%,57.86%,65.39%,65.49%
Net 30-30-30-30 E10,63.03%,91.06%,97.21%,100.00%,34.58%,35.07%,36.48%,22.04%,59.26%,66.30%,66.64%
Net 50-50-50-50 E50,68.48%,91.30%,97.88%,100.00%,32.00%,35.54%,36.89%,21.88%,59.21%,66.16%,66.63%


\begin{tabular}{llllllllllll}
\toprule
config &               1  &               2  &               3  &                4  &               5  &               6  &               7  &               8  &               9  &               10 &               11 \\
method              &                  &                  &                  &                   &                  &                  &                  &                  &                  &                  &                  \\
\midrule
KNN-10              &           71.52\% &           89.12\% &           97.12\% &  \textbackslash textbf\{100.00\%\} &           27.15\% &           30.05\% &           33.70\% &           16.83\% &           56.50\% &           63.76\% &           63.72\% \\
KNN-100             &           67.27\% &           88.52\% &           95.76\% &  \textbackslash textbf\{100.00\%\} &           30.55\% &           33.25\% &           35.34\% &           19.33\% &           58.50\% &           66.02\% & 

In [11]:
datasets = pd.DataFrame.from_dict(configs)
datasets = datasets[['max_mu', 'max_sigma', 'max_skew', 'n_vars', 'n', 'classes',]]
datasets.insert(0, "dataset", datasets.index+1)
datasets = datasets.transpose()
display(datasets)

print(datasets.to_latex())

,0,1,2,3,4,5,6,7,8,9,10
dataset,1,2,3,4,5,6,7,8,9,10,11
max_mu,2,5,5,5,1,1,1,1,1,1,1
max_sigma,2,5,5,5,10,10,10,10,10,5,5
max_skew,1,5,5,5,10,10,10,10,10,5,5
n_vars,2,5,5,50,5,5,2,5,2,2,2
n,500,10000,10000,10000,10000,100000,100000,100000,100000,100000,1000000
classes,"[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.05, 0.05, 0.05, 0.85]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.1...","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]"


\begin{tabular}{llllllllllll}
\toprule
{} &                         0 &                         1 &                         2 &                         3 &                         4 &                         5 &                         6 &                                                  7 &           8 &           9 &          10 \\
\midrule
dataset   &                         1 &                         2 &                         3 &                         4 &                         5 &                         6 &                         7 &                                                  8 &           9 &          10 &          11 \\
max\_mu    &                         2 &                         5 &                         5 &                         5 &                         1 &                         1 &                         1 &                                                  1 &           1 &           1 &           1 \\
max\_sigma &                         2 &      